https://jheck.gitbook.io/hadoop/apache-spark is a better place to for this lab with a bit updated code


# **Lab 01: Numerical summarization: PySpark RDD orientation and word count**

Look for # TODO comments in the code and replace <FILL IN> with the appropriate code.

You will find several places in the code below with the TODO comments. 

In [1]:
# The following statement will install pyspark
!pip install -q pyspark

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 48.5MB/s 


In [2]:
# To use Spark and its API import the SparkContext
from pyspark import SparkContext

# Create Spark Context
sc = SparkContext()

**Step 1: Create a base RDD**

There are two ways to create RDDs:

a) By parallelizing an existing collection in your driver program.
Parallelized collections are created by calling SparkContext’s parallelize method.  

b) By referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.


In [3]:
# We start by creating a Python List
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']

##################################################################################################################
# TODO: Generate a base RDD by using the Python list created above and the sc.parallelize method with 4 partitions
##################################################################################################################
wordsRDD =   sc.parallelize(wordsList,4)

# Print out the type of base RDD created by the above statement
print (type(wordsRDD))


<class 'pyspark.rdd.RDD'>


**Pluralize and test**

Next we add the letter 's' to each string in the base RDD we just created. 

makePlural() is a method that adds 's' to any word it receives as argument. Below we test the method with an example word: 'cat'

In [4]:
def makePlural(word):
  return word+'s'
print (makePlural('cat'))


cats


**map() operation** accepts a function as an argument. This function will be applied to each element of the source RDD and eventually will create a new RDD.
For example, we will apply makePlural() to the base RDD you created above (wordsRDD).

The end result will be a new RDD where every word of the original RDD has a 's' appended to it. 

In [5]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code 
##################################################################################################################
pluralRDD =  wordsRDD.map(makePlural)
#collect() returns all the elements of the RDD
print (pluralRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


Use map() command on wordsRDD and inside map lambda command to add "s" at the end of all the words in wordsRDD 

`Eg: RDD.map(lambda x: <function you want to perform>)`

In [6]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
pluralLambdaRDD = wordsRDD.map(lambda x: makePlural(x))
print (pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


**Lambda Function**

It is a way to create small anonymous functions, i.e. functions without a name.


Next: We will use map() and a lambda function to return the number of characters in each word and collect the result in a variable.

In [7]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
pluralLength =  pluralRDD.map(len)
print (pluralLength.collect())

[4, 9, 4, 4, 4]


**Pair RDDs**

A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD. 

We can use the lambda function with map() to return us a pair of ('<word>', 1) for every word in our base RDD.

In [8]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
wordPair = wordsRDD.map(lambda x: (x,1))
print (wordPair.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


**Counting with pair RDDs**

(1) groupByKey()  -  It groups all the elements of the RDD with the same key into a single list in one of the partitions.

For this function use wordPair


In [9]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
wordGrouped = wordPair.groupByKey()
for key, value in wordGrouped.collect():
  print( "{0} : {1}".format(key,list(value)))

cat : [1, 1]
elephant : [1]
rat : [1, 1]


In [10]:
wordCountsGrouped = wordGrouped.mapValues(lambda v: sum(v))
print(wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


**Another way of counting with pair RDDs:**

(2) reduceByKey  - The reduceByKey() transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value.

In [11]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
wordCount = wordPair.reduceByKey(lambda accum, n: accum + n)
print(wordCount.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [12]:
# Expert version: the map() to pair RDD, reduceByKey() transformation, and collect in one statement
wordCountCollected =  (wordsRDD.map(lambda x : (x,1) ).reduceByKey(lambda x, y: x+y).collect())
print(wordCountCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


Count() is used to count number of unique items in RDD

For next FILL IN use wordCount RDD 

In [13]:
##################################################################################################################
# TODO:  Replace <FILL IN> with appropriate code
##################################################################################################################
uniqueWord = wordCount.count()
print(uniqueWord)

3


FlatMap() Return a new RDD by first applying a function to all lements of this RDD, and then flattening the results.

Difference between Map and Flatmap
(look at the link for better understanding of flatmap)
https://www.linkedin.com/pulse/difference-between-map-flatmap-transformations-spark-pyspark-pandey/

In [14]:
from itertools import chain
fruits = [["apple","Hackberry"],["Jackfruit","Java Apple"],["Yangmei","Oranges"], ["Nashi Pear","Mango"] ]
#your task is to use pyspark and flatten the given list fruits
# Hint use chain in lambda 
print(sc.parallelize(fruits).flatMap(lambda x: chain(x)).collect())

['apple', 'Hackberry', 'Jackfruit', 'Java Apple', 'Yangmei', 'Oranges', 'Nashi Pear', 'Mango']


In [15]:
testString = ('my dear friend\n',
  'you don\'t know this yet but\n',
 'this lab is  going to really helpful for you with your homework\n',
 'so, thank me later\n',
 'best regards\n',
 "your well wisher\n",
 "your your your your your\n",
 "you you you you you you you you you you you",
 "this this this this",
 "homework homework homework homework homework homework homework homework homework homework ")

In [16]:
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        wordCountsCollected (str, int): An RDD consisting of (word, count) tuples.
    """
    wordCountsCollected = (wordListRDD
                       .map(lambda x: (x, 1))
                       .reduceByKey(lambda x, y: x + y))
    return wordCountsCollected

takeOrdered() : Get the N elements from a RDD ordered in ascending order or as specified by the optional key function.	

For this function you have to show top 15 words with number of word used in testString print them

Hint: use flatmap,split() and call wordCount Function

In [17]:
stringRDD = sc.parallelize(testString)
##################################################################################################################
# TODO: Replace <FILL IN> with appropriate code
##################################################################################################################
wordRDD = stringRDD.flatMap(lambda x: x.split())
top15WordsAndCounts = sc.parallelize(wordCount(wordRDD).takeOrdered(15, key=lambda x: -x[1]))
print ('\n'.join(map(lambda w: '{0}: {1}'.format(w[0], w[1]), top15WordsAndCounts.collect())))

you: 13
homework: 11
your: 7
this: 6
know: 1
yet: 1
but: 1
lab: 1
is: 1
really: 1
helpful: 1
best: 1
wisher: 1
my: 1
dear: 1
